# **Description (English):**
This is the core of your backend API. It uses FastAPI to receive prediction requests, preprocesses the input data using the saved preprocessor, then uses the trained model to make a prediction, and returns the result. This API will be deployed on a service like Render.

# **الوصف (العربية):**
هذا هو الجزء المركزي من واجهة برمجة التطبيقات (API) الخاصة بك. يستخدم إطار عمل FastAPI لاستقبال طلبات التنبؤ، ويقوم بمعالجة البيانات المدخلة باستخدام المعالج المسبق الذي تم حفظه مسبقًا، ثم يستخدم النموذج المُدرب لإجراء التنبؤ، ويعيد النتيجة. سيتم نشر هذا الـ API على خدمة مثل Render.

In [ ]:
# -*- coding: utf-8 -*-
# api_main.py

# --- 1. Import Libraries ---
# FastAPI: For building the API endpoints.
# HTTPException: To handle and return HTTP errors.
# BaseModel from pydantic: To define the structure and data types of incoming request bodies.
# joblib: To load the saved machine learning model and preprocessor.
# pandas: For data manipulation, especially to convert input data into DataFrame format.
# os: To check for file existence (model/preprocessor files).
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import pandas as pd
import os

# --- 2. Initialize FastAPI App ---
# Creates an instance of the FastAPI application.
# Includes title and description for better API documentation (accessible via /docs).
app = FastAPI(
    title="Customer Churn Prediction API",
    description="API for predicting customer churn probability based on various attributes."
)

# --- 3. Define Paths for Model Assets ---
# Specifies the filenames for the saved model and preprocessor.
# These files must be present in the same directory as this script when deployed.
MODEL_PATH = 'final_churn_predictor.pkl'
PREPROCESSOR_PATH = 'data_preprocessor.pkl'

# --- 4. Global Variables for Model and Preprocessor ---
# Declares global variables to hold the loaded model and preprocessor.
# They are initialized to None and loaded once at startup.
model = None
preprocessor = None

# --- 5. Startup Event Handler ---
# This function is executed automatically once when the FastAPI application starts up.
# It's responsible for loading the trained model and preprocessor from their .pkl files.
# This ensures that the model is ready before any prediction requests are received.
@app.on_event("startup")
async def load_assets():
    global model, preprocessor
    try:
        # Check if model files exist before attempting to load them
        if not os.path.exists(MODEL_PATH) or not os.path.exists(PREPROCESSOR_PATH):
            raise FileNotFoundError(
                f"Model files not found. Ensure '{MODEL_PATH}' and '{PREPROCESSOR_PATH}' are in the same directory."
                f"Current directory: {os.getcwd()}"
            )
        model = joblib.load(MODEL_PATH)
        preprocessor = joblib.load(PREPROCESSOR_PATH)
        print("Model and preprocessor loaded successfully!")
    except Exception as e:
        print(f"CRITICAL ERROR: Failed to load model or preprocessor: {e}")
        # In a production environment, you might want the application to exit if essential assets fail to load.

# --- 6. Define Input Data Schema (Pydantic BaseModel) ---
# This class defines the expected structure and data types of the JSON payload
# that the API will receive for a churn prediction request.
# The attribute names must exactly match the original features of your dataset.
class CustomerData(BaseModel):
    gender: str
    SeniorCitizen: int
    Partner: str
    Dependents: str
    tenure: int
    PhoneService: str
    MultipleLines: str
    InternetService: str
    OnlineSecurity: str
    OnlineBackup: str
    DeviceProtection: str
    TechSupport: str
    StreamingTV: str
    StreamingMovies: str
    Contract: str
    PaperlessBilling: str
    PaymentMethod: str
    MonthlyCharges: float
    TotalCharges: float

# --- 7. Root Endpoint ---
# A simple GET endpoint to confirm the API is running.
# Accessible at the base URL (e.g., https://your-api-url.onrender.com/).
@app.get("/", summary="Root Endpoint")
def read_root():
    return {"message": "Welcome to Customer Churn Prediction API! Visit /docs for API documentation."}

# --- 8. Prediction Endpoint ---
# A POST endpoint where clients (like your Streamlit app) will send customer data
# to get a churn prediction.
# It takes a CustomerData object as input, preprocesses it, makes a prediction, and returns the probability.
@app.post("/predict_churn/", summary="Predict Customer Churn Probability")
async def predict_churn(data: CustomerData):
    # Check if model and preprocessor are loaded, if not, raise an internal server error.
    if model is None or preprocessor is None:
        raise HTTPException(status_code=500, detail="Model or preprocessor not loaded. Server issue.")

    # Convert the incoming Pydantic model data into a Pandas DataFrame.
    # This is crucial because the preprocessor and model expect DataFrame input.
    input_df = pd.DataFrame([data.dict()])

    # Re-calculate Engineered Features:
    # It's critically important to re-create the *exact same* engineered features
    # in the API that were created during the model training phase.
    # The model expects these features in its input.
    input_df['TotalServices'] = (input_df[['PhoneService', 'MultipleLines', 'InternetService',
                                           'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                                           'TechSupport', 'StreamingTV', 'StreamingMovies']] == 'Yes').sum(axis=1)
    input_df['MonthlyChargePerTenure'] = input_df['MonthlyCharges'] / (input_df['tenure'] + 1e-6)
    input_df['HasInternetService'] = input_df['InternetService'].apply(lambda x: 1 if x != 'No' else 0)
    input_df['HasMultipleLines'] = input_df['MultipleLines'].apply(lambda x: 1 if x == 'Yes' else 0)
    input_df['IsSeniorCitizen_Married'] = input_df.apply(lambda row: 1 if row['SeniorCitizen'] == 1 and row['Partner'] == 'Yes' else 0, axis=1)

    # Apply the Preprocessor:
    # Use 'transform' method, NOT 'fit_transform', as the preprocessor has already been fitted during training.
    # This transforms raw input features into the scaled/encoded format expected by the model.
    try:
        processed_input = preprocessor.transform(input_df)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error during data preprocessing: {e}. Check input data format and types.")

    # Make the Prediction:
    # Get the churn probability. model.predict_proba returns probabilities for both classes (0 and 1).
    # We take the probability of class 1 (churn).
    churn_probability = model.predict_proba(processed_input)[:, 1][0]

    # Return the prediction result as a JSON response.
    return {"churn_probability": churn_probability.item()} # .item() converts numpy float to standard Python float